In [ ]:
%pip install llama-index-llms-openai pandas[jinja2] spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 1.7 MB/s eta 0:00:00
   ━━━━

In [ ]:
pip install llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.9 MB/s eta 0:00:00


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.core import (
    TreeIndex,
    VectorStoreIndex,
    SimpleDirectoryReader,
    Response,Document
)
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import RelevancyEvaluator
from llama_index.core.node_parser import SentenceSplitter,SimpleNodeParser
import pandas as pd


pd.set_option("display.max_colwidth", 0)

In [ ]:
import openai
# Configurar la API de OpenAI con tu clave
openai.api_key = 'your-token'

# Configurar el modelo GPT-4 Mini
gpt4mini_model = "gpt-4o-mini"

In [ ]:
# gpt-4
gpt4 = OpenAI(temperature=0, model=gpt4mini_model)
evaluator = RelevancyEvaluator(llm=gpt4)

In [ ]:
#Seleccionamos el corpus correspondiente
corpus = pd.read_csv('respuestas_gpt35.csv')
corpus = pd.read_csv('respuestas_gpt4.csv')
corpus = pd.read_csv('respuestas_mixtral.csv')

In [ ]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [ ]:
def relevancy_funcion(contexto, pregunta):

  splitter = SentenceSplitter(chunk_size=512)
  chunks = splitter.split_text(contexto)
  # Crear documentos a partir de los chunks
  documents = [Document(text=chunk) for chunk in chunks]

  # Crear el índice vectorial a partir de los documentos
  vector_index = VectorStoreIndex.from_documents(documents)
  query_engine = vector_index.as_query_engine()

  response_vector = query_engine.query(pregunta)

  eval_result = evaluator.evaluate_response(query=pregunta,response=response_vector)

  return(eval_result.passing,eval_result.score)

In [ ]:
corpus[['Relevancy_passing','Relevancy_score']] = corpus.apply(lambda x:pd.Series(relevancy_funcion(str(x['frase']),str(x['pregunta_generada'])))  , axis = 1)

In [ ]:
corpus.to_csv('Score_relevancy_mistral.csv', index=False)